In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import warnings

warnings.filterwarnings('ignore')

# Enhanced Academic publication style settings
plt.style.use('seaborn-v0_8-paper')
sns.set_context("paper", font_scale=1.6)
sns.set_palette("deep")

# Configure matplotlib for publication quality with BOLD and BIGGER fonts
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']
plt.rcParams['font.size'] = 13
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.titlesize'] = 17
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 13
plt.rcParams['ytick.labelsize'] = 13
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['legend.frameon'] = True
plt.rcParams['legend.edgecolor'] = 'black'
plt.rcParams['legend.title_fontsize'] = 13
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['xtick.major.width'] = 1.2
plt.rcParams['ytick.major.width'] = 1.2

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.precision', 4)

print("Libraries loaded successfully!")
print("Enhanced academic publication style configured")
print("Experiment 2: WITH Artist Features (414 features)")

In [ ]:
# Load Experiment 2 results (with artist features)
df_raw = pd.read_csv('../results/metrics/experiment2_with_artist/enhanced_results_detailed_20251231_143812.csv')

# Filter to keep only specified models
selected_models = [
    'CatBoost', 'CatBoost_tuned',
    'LightGBM', 'LightGBM_tuned',
    'XGBoost', 'XGBoost_tuned',
    'ExtraTrees', 'ExtraTrees_tuned',
    'MLPRegressor', 'MLPRegressor_tuned',
    'RandomForest', 'RandomForest_tuned'
]

df = df_raw[df_raw['model'].isin(selected_models)].copy()

print(f"Loaded {len(df_raw)} total model results from Experiment 2")
print(f"Filtered to {len(df)} results ({len(selected_models)} models)")
print(f"Targets: {df['target'].unique().tolist()}")
print(f"Selected Models: {sorted(df['model'].unique().tolist())}")
print(f"Features: 414 (23 audio + 5 text + 2 sentiment + 384 embeddings)")
print(f"Artist Features: log_total_artist_followers + avg_artist_popularity")

---
## 🏆 Best Models Summary (Quick Overview)

In [ ]:
# Find best model for each target
best_models = df.loc[df.groupby('target')['r2'].idxmax()][['target', 'model', 'r2', 'rmse', 'mae']]
best_models = best_models.sort_values('r2', ascending=False)

styled_best = best_models.style.format({
    'r2': '{:.4f}',
    'rmse': '{:.4f}',
    'mae': '{:.4f}'
}).set_caption('Best Model Per Target').set_table_styles([
    {'selector': 'caption', 'props': [('font-size', '18px'), ('font-weight', 'bold'), ('color', '#2E86AB')]}
])

display(styled_best)

---
## 🎭 Valence Models (Emotional Positivity)

In [ ]:
def create_styled_table(df, target, sort_by='r2', ascending=False):
    """Create a beautifully styled table for a specific target, separated by default and tuned"""
    target_df = df[df['target'] == target].copy()
    target_df = target_df.sort_values(sort_by, ascending=ascending)
    
    # Select key columns
    cols = ['model', 'r2', 'rmse', 'mae', 'explained_variance', 'train_time_seconds']
    target_df = target_df[cols].reset_index(drop=True)
    
    # Split into default and tuned
    default_df = target_df[~target_df['model'].str.contains('_tuned')].copy()
    tuned_df = target_df[target_df['model'].str.contains('_tuned')].copy()
    
    # Add ranks
    default_df['rank'] = range(1, len(default_df) + 1)
    tuned_df['rank'] = range(1, len(tuned_df) + 1)
    
    default_df = default_df[['rank'] + cols]
    tuned_df = tuned_df[['rank'] + cols]
    
    # Display both tables
    print(f"\n{'='*80}")
    print(f"{target.upper()} - DEFAULT MODELS")
    print('='*80)
    
    styled_default = default_df.style.format({
        'r2': '{:.4f}',
        'rmse': '{:.4f}',
        'mae': '{:.4f}',
        'explained_variance': '{:.4f}',
        'train_time_seconds': '{:.1f}s'
    }).set_caption(
        f'{target.upper()} - Default Models (Sorted by R²)'
    ).set_table_styles([
        {'selector': 'caption', 'props': [('font-size', '14px'), ('font-weight', 'bold'), ('color', '#2E86AB')]},
        {'selector': 'th', 'props': [('background-color', '#4A90D9'), ('color', 'white')]},
    ])
    display(styled_default)
    
    print(f"\n{'='*80}")
    print(f"{target.upper()} - TUNED MODELS")
    print('='*80)
    
    styled_tuned = tuned_df.style.format({
        'r2': '{:.4f}',
        'rmse': '{:.4f}',
        'mae': '{:.4f}',
        'explained_variance': '{:.4f}',
        'train_time_seconds': '{:.1f}s'
    }).set_caption(
        f'{target.upper()} - Tuned Models (Sorted by R²)'
    ).set_table_styles([
        {'selector': 'caption', 'props': [('font-size', '14px'), ('font-weight', 'bold'), ('color', '#F18F01')]},
        {'selector': 'th', 'props': [('background-color', '#F18F01'), ('color', 'white')]},
    ])
    display(styled_tuned)

create_styled_table(df, 'valence')

---
## ⚡ Energy Models (Intensity)

In [ ]:
display(create_styled_table(df, 'energy'))

---
## 💃 Danceability Models

In [ ]:
display(create_styled_table(df, 'danceability'))

---
## 🌟 Popularity Models (Hardest Target)

In [ ]:
display(create_styled_table(df, 'popularity'))

---
## 📈 Visual Comparison: R² Scores Across All Targets

In [ ]:
# Create separate R² comparison heatmaps for default and tuned models - Enhanced styling
df_default = df[~df['model'].str.contains('_tuned')].copy()
df_tuned = df[df['model'].str.contains('_tuned')].copy()

# Clean model names for tuned (remove _tuned suffix)
df_tuned['model_clean'] = df_tuned['model'].str.replace('_tuned', '')

# Create pivots
pivot_default = df_default.pivot(index='model', columns='target', values='r2')
pivot_tuned = df_tuned.pivot(index='model_clean', columns='target', values='r2')

# Order columns
target_order = ['valence', 'energy', 'danceability', 'popularity']
pivot_default = pivot_default[target_order]
pivot_tuned = pivot_tuned[target_order]

# Sort by average R²
pivot_default['avg_r2'] = pivot_default.mean(axis=1)
pivot_default = pivot_default.sort_values('avg_r2', ascending=False).drop('avg_r2', axis=1)

pivot_tuned['avg_r2'] = pivot_tuned.mean(axis=1)
pivot_tuned = pivot_tuned.sort_values('avg_r2', ascending=False).drop('avg_r2', axis=1)

# Create side-by-side heatmaps with enhanced styling
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Default models

sns.heatmap(pivot_default, annot=True, fmt='.3f', cmap='RdYlGn', center=0.3,
            linewidths=2, linecolor='white', ax=axes[0], vmin=-0.2, vmax=0.9,
            cbar_kws={'shrink': 0.85},
            annot_kws={'fontsize': 12, 'weight': 'bold'})
axes[0].set_title('Default Models - R² Scores', fontsize=17, fontweight='bold', pad=20)
axes[0].set_xlabel('Target', fontsize=15, fontweight='bold', labelpad=10)
axes[0].set_ylabel('Model', fontsize=15, fontweight='bold', labelpad=10)
axes[0].tick_params(axis='x', labelsize=14, width=1.5)
axes[0].tick_params(axis='y', labelsize=13, width=1.5)
plt.setp(axes[0].get_xticklabels(), fontweight='bold')
plt.setp(axes[0].get_yticklabels(), fontweight='bold')

# Tuned models
sns.heatmap(pivot_tuned, annot=True, fmt='.3f', cmap='RdYlGn', center=0.3,
            linewidths=2, linecolor='white', ax=axes[1], vmin=-0.2, vmax=0.9,
            cbar_kws={'shrink': 0.85},
            annot_kws={'fontsize': 12, 'weight': 'bold'})
axes[1].set_title('Tuned Models - R² Scores', fontsize=17, fontweight='bold', pad=20)
axes[1].set_xlabel('Target', fontsize=15, fontweight='bold', labelpad=10)
axes[1].set_ylabel('Model', fontsize=15, fontweight='bold', labelpad=10)
axes[1].tick_params(axis='x', labelsize=14, width=1.5)
axes[1].tick_params(axis='y', labelsize=13, width=1.5)
plt.setp(axes[1].get_xticklabels(), fontweight='bold')
plt.setp(axes[1].get_yticklabels(), fontweight='bold')

# plt.suptitle('R² Scores Comparison', 
#              fontsize=19, fontweight='bold', y=0.98)
plt.tight_layout()
plt.savefig('../results/figures/experiment2_r2_heatmap_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: results/figures/experiment2_r2_heatmap_comparison.png")

---
## 📊 Top 5 Models Bar Chart Per Target

In [ ]:
# Create separate bar charts for default and tuned models - Enhanced styling
targets = ['valence', 'energy', 'danceability', 'popularity']

# Default models
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
for ax, target in zip(axes.flatten(), targets):
    top_models = df_default[df_default['target'] == target].nlargest(6, 'r2')
    
    bars = ax.barh(top_models['model'], top_models['r2'], color='#4A90D9', 
                   edgecolor='black', alpha=0.85, linewidth=1.5)
    ax.set_xlabel('R² Score', fontsize=14, fontweight='bold', labelpad=8)
    ax.set_title(f'{target.upper()}', fontsize=15, fontweight='bold', pad=15)
    ax.set_xlim(0, max(top_models['r2']) * 1.15)
    ax.grid(True, alpha=0.3, axis='x', linestyle='--', linewidth=1.2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(1.5)
    ax.spines['bottom'].set_linewidth(1.5)
    ax.tick_params(axis='both', width=1.2, labelsize=12)
    
    # Add value labels
    for bar, val in zip(bars, top_models['r2']):
        ax.text(val + 0.01, bar.get_y() + bar.get_height()/2, f'{val:.3f}', 
                va='center', fontsize=11, fontweight='bold')
    
    ax.invert_yaxis()
    plt.setp(ax.get_yticklabels(), fontweight='bold', fontsize=12)

plt.suptitle('Default Models', 
             fontsize=17, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('../results/figures/experiment2_default_models.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: results/figures/experiment2_default_models.png")

# Tuned models
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
for ax, target in zip(axes.flatten(), targets):
    top_models = df_tuned[df_tuned['target'] == target].nlargest(6, 'r2')
    
    bars = ax.barh(top_models['model'], top_models['r2'], color='#F18F01', 
                   edgecolor='black', alpha=0.85, linewidth=1.5)
    ax.set_xlabel('R² Score', fontsize=14, fontweight='bold', labelpad=8)
    ax.set_title(f'{target.upper()}', fontsize=15, fontweight='bold', pad=15)
    ax.set_xlim(0, max(top_models['r2']) * 1.15)
    ax.grid(True, alpha=0.3, axis='x', linestyle='--', linewidth=1.2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(1.5)
    ax.spines['bottom'].set_linewidth(1.5)
    ax.tick_params(axis='both', width=1.2, labelsize=12)
    
    # Add value labels
    for bar, val in zip(bars, top_models['r2']):
        ax.text(val + 0.01, bar.get_y() + bar.get_height()/2, f'{val:.3f}', 
                va='center', fontsize=11, fontweight='bold')
    
    ax.invert_yaxis()
    plt.setp(ax.get_yticklabels(), fontweight='bold', fontsize=12)

plt.suptitle('Tuned Models', 
             fontsize=17, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('../results/figures/experiment2_tuned_models.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: results/figures/experiment2_tuned_models.png")

---
## ⏱️ Training Time Analysis

In [ ]:
# Aggregate training time by model - Enhanced styling
time_by_model = df.groupby('model')['train_time_seconds'].mean().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(14, 10))
colors = plt.cm.viridis(np.linspace(0, 1, len(time_by_model)))
bars = ax.barh(time_by_model.index, time_by_model.values, color=colors, 
               edgecolor='black', alpha=0.85, linewidth=1.5)
ax.set_xlabel('Average Training Time (seconds)', fontsize=15, fontweight='bold', labelpad=10)
ax.set_title('Average Training Time by Model', fontsize=17, fontweight='bold', pad=20)
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x', linestyle='--', linewidth=1.2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(1.5)
ax.spines['bottom'].set_linewidth(1.5)
ax.tick_params(axis='both', width=1.2, labelsize=12)

# Add value labels
for bar, val in zip(bars, time_by_model.values):
    if val > 100:
        label = f'{val/60:.1f}m'
    else:
        label = f'{val:.1f}s'
    ax.text(val + 10, bar.get_y() + bar.get_height()/2, label, 
            va='center', fontsize=11, fontweight='bold')

plt.setp(ax.get_yticklabels(), fontweight='bold', fontsize=12)
plt.tight_layout()
plt.savefig('../results/figures/experiment2_training_time_selected.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: results/figures/experiment2_training_time_selected.png")

---
## 📉 RMSE Comparison

In [ ]:
# Create separate RMSE comparison for default and tuned models - Enhanced styling
targets = ['valence', 'energy', 'danceability', 'popularity']

# Default models RMSE
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
for ax, target in zip(axes.flatten(), targets):
    top_models = df_default[df_default['target'] == target].nsmallest(6, 'rmse')
    
    bars = ax.barh(top_models['model'], top_models['rmse'], color='#60276d', 
                   edgecolor='black', alpha=0.85, linewidth=1.5)
    ax.set_xlabel('RMSE', fontsize=14, fontweight='bold', labelpad=8)
    ax.set_title(f'{target.upper()}', fontsize=15, fontweight='bold', pad=15)
    ax.grid(True, alpha=0.3, axis='x', linestyle='--', linewidth=1.2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(1.5)
    ax.spines['bottom'].set_linewidth(1.5)
    ax.tick_params(axis='both', width=1.2, labelsize=12)
    
    # Add value labels
    for bar, val in zip(bars, top_models['rmse']):
        ax.text(val + 0.001, bar.get_y() + bar.get_height()/2, f'{val:.4f}', 
                va='center', fontsize=11, fontweight='bold')
    
    ax.invert_yaxis()
    plt.setp(ax.get_yticklabels(), fontweight='bold', fontsize=12)

plt.suptitle('Default Models', 
             fontsize=17, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('../results/figures/experiment2_default_rmse.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: results/figures/experiment2_default_rmse.png")

# Tuned models RMSE
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
for ax, target in zip(axes.flatten(), targets):
    top_models = df_tuned[df_tuned['target'] == target].nsmallest(6, 'rmse')
    
    bars = ax.barh(top_models['model'], top_models['rmse'], color='#F18F01', 
                   edgecolor='black', alpha=0.85, linewidth=1.5)
    ax.set_xlabel('RMSE', fontsize=14, fontweight='bold', labelpad=8)
    ax.set_title(f'{target.upper()}', fontsize=15, fontweight='bold', pad=15)
    ax.grid(True, alpha=0.3, axis='x', linestyle='--', linewidth=1.2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(1.5)
    ax.spines['bottom'].set_linewidth(1.5)
    ax.tick_params(axis='both', width=1.2, labelsize=12)
    
    # Add value labels
    for bar, val in zip(bars, top_models['rmse']):
        ax.text(val + 0.001, bar.get_y() + bar.get_height()/2, f'{val:.4f}', 
                va='center', fontsize=11, fontweight='bold')
    
    ax.invert_yaxis()
    plt.setp(ax.get_yticklabels(), fontweight='bold', fontsize=12)

plt.suptitle('Tuned Models', 
             fontsize=17, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('../results/figures/experiment2_tuned_rmse.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: results/figures/experiment2_tuned_rmse.png")

---
## 🔬 Default vs Tuned Models Comparison

In [ ]:
# Compare default vs tuned versions
tuned_models = df[df['model'].str.contains('_tuned')].copy()
tuned_models['base_model'] = tuned_models['model'].str.replace('_tuned', '')

default_models = df[~df['model'].str.contains('_tuned')].copy()
default_models['base_model'] = default_models['model']

# Merge to compare
comparison = tuned_models.merge(
    default_models[['target', 'base_model', 'r2', 'rmse']], 
    on=['target', 'base_model'], 
    suffixes=('_tuned', '_default')
)
comparison['r2_improvement'] = comparison['r2_tuned'] - comparison['r2_default']
comparison['rmse_improvement'] = comparison['rmse_default'] - comparison['rmse_tuned']  # Positive = better

# Display improvement summary
improvement_summary = comparison.groupby('base_model').agg({
    'r2_improvement': 'mean',
    'rmse_improvement': 'mean'
}).sort_values('r2_improvement', ascending=False)

print("Tuning Impact (Average across all targets):")
print("="*50)
styled_improvement = improvement_summary.style.format({
    'r2_improvement': '{:+.4f}',
    'rmse_improvement': '{:+.4f}'
}).set_caption(
    'Tuning Impact: Positive = Improvement'
).set_table_styles([
    {'selector': 'caption', 'props': [('font-size', '16px'), ('font-weight', 'bold')]},
])
display(styled_improvement)

In [ ]:
# Visualize tuning impact - Enhanced styling
fig, ax = plt.subplots(figsize=(14, 7))

x = np.arange(len(improvement_summary))
width = 0.6

bars = ax.bar(x, improvement_summary['r2_improvement'], width, 
              color=['#90EE90' if v > 0 else '#FF6B6B' for v in improvement_summary['r2_improvement']],
              edgecolor='black', alpha=0.85, linewidth=1.5)

ax.set_xlabel('Model', fontsize=15, fontweight='bold', labelpad=10)
ax.set_ylabel('R² Improvement (Tuned - Default)', fontsize=15, fontweight='bold', labelpad=10)
ax.set_title('Hyperparameter Tuning Impact on R²', 
             fontsize=17, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(improvement_summary.index, rotation=45, ha='right', fontweight='bold', fontsize=12)
ax.axhline(y=0, color='black', linestyle='-', linewidth=2)
ax.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=1.2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(1.5)
ax.spines['bottom'].set_linewidth(1.5)
ax.tick_params(axis='both', width=1.2, labelsize=12)

# Add value labels
for bar, val in zip(bars, improvement_summary['r2_improvement']):
    y_pos = val + 0.003 if val >= 0 else val - 0.012
    ax.text(bar.get_x() + bar.get_width()/2, y_pos, f'{val:+.3f}', 
            ha='center', va='bottom' if val >= 0 else 'top', 
            fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('../results/figures/experiment2_tuning_impact_selected.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: results/figures/experiment2_tuning_impact_selected.png")

---
## 📋 Complete Results Table (All Models, All Targets)

In [ ]:
# Create comprehensive tables separated by default and tuned
full_table = df[['target', 'model', 'r2', 'rmse', 'mae', 'explained_variance', 
                 'pred_mean', 'pred_std', 'residual_mean', 'residual_std', 'train_time_seconds']].copy()

# Default models
default_table = full_table[~full_table['model'].str.contains('_tuned')].copy()
default_table = default_table.sort_values(['target', 'r2'], ascending=[True, False])

# Tuned models
tuned_table = full_table[full_table['model'].str.contains('_tuned')].copy()
tuned_table = tuned_table.sort_values(['target', 'r2'], ascending=[True, False])

def color_by_target(val):
    colors = {
        'valence': 'background-color: #E8F4FD',
        'energy': 'background-color: #FDF4E8',
        'danceability': 'background-color: #F4E8FD',
        'popularity': 'background-color: #E8FDF4'
    }
    return colors.get(val, '')

print("="*80)
print("DEFAULT MODELS - COMPLETE RESULTS")
print("="*80)

styled_default = default_table.style.format({
    'r2': '{:.4f}',
    'rmse': '{:.4f}',
    'mae': '{:.4f}',
    'explained_variance': '{:.4f}',
    'pred_mean': '{:.4f}',
    'pred_std': '{:.4f}',
    'residual_mean': '{:.4f}',
    'residual_std': '{:.4f}',
    'train_time_seconds': '{:.1f}'
}).applymap(color_by_target, subset=['target']).set_caption(
    'Default Models - Complete Results'
).set_table_styles([
    {'selector': 'caption', 'props': [('font-size', '16px'), ('font-weight', 'bold')]},
    {'selector': 'th', 'props': [('background-color', '#4A90D9'), ('color', 'white')]},
])

display(styled_default)

print("\n" + "="*80)
print("TUNED MODELS - COMPLETE RESULTS")
print("="*80)

styled_tuned = tuned_table.style.format({
    'r2': '{:.4f}',
    'rmse': '{:.4f}',
    'mae': '{:.4f}',
    'explained_variance': '{:.4f}',
    'pred_mean': '{:.4f}',
    'pred_std': '{:.4f}',
    'residual_mean': '{:.4f}',
    'residual_std': '{:.4f}',
    'train_time_seconds': '{:.1f}'
}).applymap(color_by_target, subset=['target']).set_caption(
    'Tuned Models - Complete Results'
).set_table_styles([
    {'selector': 'caption', 'props': [('font-size', '16px'), ('font-weight', 'bold')]},
    {'selector': 'th', 'props': [('background-color', '#F18F01'), ('color', 'white')]},
])

display(styled_tuned)

---
## 🎯 Final Recommendations

In [ ]:
# Get top 2 models per target for both default and tuned
recommendations = []
targets = ['valence', 'energy', 'danceability', 'popularity']

print("="*80)
print("FINAL RECOMMENDATIONS")
print("="*80)

for target in targets:
    print(f"\n{target.upper()}:")
    print("-" * 60)
    
    # Top default model
    top_default = df_default[df_default['target'] == target].nlargest(1, 'r2').iloc[0]
    print(f"  Default: {top_default['model']}")
    print(f"    R² = {top_default['r2']:.4f} | RMSE = {top_default['rmse']:.4f} | Time = {top_default['train_time_seconds']:.1f}s")
    
    # Top tuned model
    top_tuned = df_tuned[df_tuned['target'] == target].nlargest(1, 'r2').iloc[0]
    print(f"  Tuned:   {top_tuned['model']}")
    print(f"    R² = {top_tuned['r2']:.4f} | RMSE = {top_tuned['rmse']:.4f} | Time = {top_tuned['train_time_seconds']:.1f}s")
    
    # Improvement
    r2_imp = top_tuned['r2'] - top_default['r2']
    rmse_imp = top_default['rmse'] - top_tuned['rmse']
    print(f"  Improvement: R² {r2_imp:+.4f} | RMSE {rmse_imp:+.4f}")
    
    # Add to recommendations
    rec_default = top_default[['target', 'model', 'r2', 'rmse', 'train_time_seconds']].to_dict()
    rec_default['type'] = 'Default'
    recommendations.append(rec_default)
    
    rec_tuned = top_tuned[['target', 'model', 'r2', 'rmse', 'train_time_seconds']].to_dict()
    rec_tuned['type'] = 'Tuned'
    recommendations.append(rec_tuned)

# Create recommendations dataframe
rec_df = pd.DataFrame(recommendations)
rec_df = rec_df[['target', 'type', 'model', 'r2', 'rmse', 'train_time_seconds']]

print("\n" + "="*80)
print("RECOMMENDATIONS TABLE")
print("="*80)

styled_rec = rec_df.style.format({
    'r2': '{:.4f}',
    'rmse': '{:.4f}',
    'train_time_seconds': '{:.1f}s'
}).set_caption('🎯 Final Model Recommendations').set_table_styles([
    {'selector': 'caption', 'props': [('font-size', '18px'), ('font-weight', 'bold'), ('color', '#2E86AB')]},
    {'selector': 'th', 'props': [('background-color', '#2E86AB'), ('color', 'white')]},
])

display(styled_rec)